In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import copy

In [ ]:
import numpy as np
from numpy import vstack
from numpy import random
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from random import sample

In [ ]:
torch.cuda.is_available()

True

Parameters


In [ ]:
LEARNING_RATE = 0.001

NUM_EPOCHS = 300
NUM_RAND_UNK = 5000
EMBEDDING_DIM = 1024
HIDDEN_DIM = 256

AUG_NUM_EPOCHS = 71
AUG_NUM_RAND_UNK = 20000
AUG_EMBEDDING_DIM = 1024
AUG_HIDDEN_DIM = 256

Import the data

In [ ]:
import re
# Use train.tag file and train.word file to create training data
training_data = []
tags = []
sentences = []
with open("train6.tag") as tagsData:
  with open("train6.word") as sentencesData:
    for line in tagsData:
      line = re.sub(r'\bM\b', 'NN', line) # Group (M) under (NN)
      line = re.sub(r'\bRPN\b', 'PRO', line) # Group (RPN) under (PRO)
      line = re.sub(r'\bCUR\b|\bDBL\b|\bETC\b|\bKAN\b','SYM', line) # Group (CUR), (DBL), (ETC), (KAN) under (SYM)
      line = re.sub(r'\bUH\b', 'PA', line) # Group (UH) under (PA)
      line = re.sub(r'\bVB_JJ\b|\bV_COM\b|\bVCOM\b', 'VB', line) # Group (VB_JJ), (VB_COM) under (VB)
      tags.append(line)
    for line in sentencesData:
      sentences.append(line)

for i in range(len(tags)):
  pair = (sentences[i].split(), tags[i].split())
  training_data.append(pair)

In [ ]:

import sklearn.model_selection as model_selection

X = []
y = []
for item in training_data:
  X.append(item[0])
  y.append(item[1])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.9,test_size=0.1, random_state=21)

# create training_data and test_data from this (mostly so I don't have to change my code a ton)
training_data = []
test_data = []

for i in range(len(X_train)):
  item = [X_train[i], y_train[i]]
  training_data.append(item)

for i in range(len(X_test)):
  item = [X_test[i], y_test[i]]
  test_data.append(item)


Import close-test and open-test data

In [ ]:
close_test_data = []
tags = []
sentences = []
with open("CLOSE-TEST.tag") as tagsData:
  with open("CLOSE-TEST.word") as sentencesData:
    for line in tagsData:
      line = re.sub(r'\bM\b', 'NN', line) # Group (M) under (NN)
      line = re.sub(r'\bRPN\b', 'PRO', line) # Group (RPN) under (PRO)
      line = re.sub(r'\bCUR\b|\bDBL\b|\bETC\b|\bKAN\b','SYM', line) # Group (CUR), (DBL), (ETC), (KAN) under (SYM)
      line = re.sub(r'\bUH\b', 'PA', line) # Group (UH) under (PA)
      line = re.sub(r'\bVB_JJ\b|\bV_COM\b|\bVCOM\b', 'VB', line) # Group (VB_JJ), (VB_COM) under (VB)
      tags.append(line)
    for line in sentencesData:
      sentences.append(line)

for i in range(len(tags)):
  pair = (sentences[i].split(), tags[i].split())
  close_test_data.append(pair)

In [ ]:
open_test_data = []
tags = []
sentences = []
with open("OPEN-TEST.tag") as tagsData:
  with open("OPEN-TEST.word") as sentencesData:
    for line in tagsData:
      line = re.sub(r'\bM\b', 'NN', line) # Group (M) under (NN)
      line = re.sub(r'\bRPN\b', 'PRO', line) # Group (RPN) under (PRO)
      line = re.sub(r'\bCUR\b|\bDBL\b|\bETC\b|\bKAN\b','SYM', line) # Group (CUR), (DBL), (ETC), (KAN) under (SYM)
      line = re.sub(r'\bUH\b', 'PA', line) # Group (UH) under (PA)
      line = re.sub(r'\bVB_JJ\b|\bV_COM\b|\bVCOM\b', 'VB', line) # Group (VB_JJ), (VB_COM) under (VB)
      tags.append(line)
    for line in sentencesData:
      sentences.append(line)

for i in range(len(tags)):
  pair = (sentences[i].split(), tags[i].split())
  open_test_data.append(pair)

In [ ]:
print(open_test_data)

[(['លោក~ស្រី', 'ឃុន', 'វត្តី', 'ស្រី', 'រាជ្យនី'], ['PRO', 'PN', 'PN', 'PN', 'PN']), (['គាត់', 'ឈ្មោះ', 'មឿន', 'តឿប'], ['PRO', 'NN', 'PN', 'PN']), (['លោក', 'ឃូ', 'ប៉ាវ', 'ស្រ៊ុន'], ['PRO', 'PN', 'PN', 'PN']), (['លោក~ជំទាវ', 'សុខ', 'អាន'], ['PRO', 'PN', 'PN']), (['ឯក_ឧត្ដម', 'ពិសាល'], ['PRO', 'PN']), (['គាត់', 'ឈ្មោះ', 'ម៉ៅ', 'វ៉ាន់និត'], ['PRO', 'NN', 'PN', 'PN']), (['កញ្ញា', 'ហាត់', 'ទូច'], ['PRO', 'PN', 'PN']), (['២', '.', '១', 'ប្រភព', 'ទាំងឡាយ', 'របស់', 'គម្រោង'], ['CD', 'SYM', 'CD', 'NN', 'DT', 'IN', 'NN']), (['កញ្ញា', 'យៀម', 'ស្រី', 'លាក់'], ['PRO', 'PN', 'PN', 'PN']), (['អ្នក~នាង', 'ជា', 'និស្ស័យ'], ['PRO', 'PN', 'PN']), (['នេះ', 'គឺ_ជា', 'បញ្ញា'], ['DT', 'VB', 'PN']), (['លោក', 'ឃឹម'], ['PRO', 'PN']), (['លោក~ស្រី', 'សុខ', 'លីន'], ['PRO', 'PN', 'PN']), (['ខ្ញុំ', 'ឈ្មោះ', 'សាម៉ៃ'], ['PRO', 'NN', 'PN']), (['លោក~ស្រី', 'អ៊ីសេង'], ['PRO', 'PN']), (['លោក', 'បាន', 'បន្ថែម', 'ថា', '៖', '«', 'បើ', 'ប្រៀប_ធៀប', 'នឹង', 'កាល_ពី', 'មុន', 'វា', 'មាន', 'ការ~ធ្លាក់_ចុះ', 'ចំនួន', 'កុមារ', 'នៅ_

<h1>Create Augmented Data</h1>
<p>This data is made from the <code>training_set</code> of the original 12,000 sentence dataset. This is so that the model is completely blind to the <code>test_set</code>.

Bucket by class (POS)

In [ ]:
buckets = {}

for i in range(len(training_data)):
  for j in range(len(training_data[i][0])):
    if training_data[i][1][j] not in buckets:
      buckets[training_data[i][1][j]] = []
    buckets[training_data[i][1][j]].append(training_data[i][0][j])

In [ ]:
total = 0
for key in buckets:
  total = total + len(buckets[key])
print(total)

115942


In [ ]:
for key in buckets:
  print("{0}: {1}, {2:.2f}%".format(key, len(buckets[key]), len(buckets[key]) / total * 100))

NN: 28933, 24.95%
AUX: 2209, 1.91%
VB: 16718, 14.42%
JJ: 4035, 3.48%
CD: 3009, 2.60%
IN: 12035, 10.38%
PN: 18066, 15.58%
SYM: 5236, 4.52%
PRO: 12561, 10.83%
DT: 3894, 3.36%
RB: 5773, 4.98%
CC: 2519, 2.17%
PA: 819, 0.71%
QT: 74, 0.06%
AB: 61, 0.05%


Contain all the possible words for each POS

In [ ]:
bucketsSets = {}
for key in buckets:
  bucketsSets[key] = list(set(buckets[key]))

In [ ]:
for key in bucketsSets:
  print("{0}: {1}".format(key, len(bucketsSets[key])))

NN: 3050
AUX: 6
VB: 1469
JJ: 587
CD: 322
IN: 182
PN: 1952
SYM: 25
PRO: 80
DT: 36
RB: 327
CC: 43
PA: 49
QT: 1
AB: 12


Switch out the words of one POS for another word of that POS

In [ ]:
def createAugmentedDataForOneClass(POS, percent):
  augmented_data = copy.deepcopy(training_data)
  new_data = []
  for i in range(len(augmented_data)):
    POSList = augmented_data[i][1]
    if POS in POSList:
      for j in range(len(POSList)):
        if POSList[j] == POS:
          index = random.randint(len(buckets[POS]))
          augmented_data[i][0][j] = buckets[POS][index]
          new_data.append(augmented_data[i])

  return sample(new_data, int(float(len(new_data))*percent))

In [ ]:
augmented_data = []

parts_of_speech = ['AB','AUX','CC','CD','DT','IN','JJ','VB','NN','PN','PA','PRO','QT','RB','SYM']
for pos in parts_of_speech:
  new_data = createAugmentedDataForOneClass(pos, 0.1)
  for sentence in new_data:
    augmented_data.append(sentence)

print(len(augmented_data))

11586


Write out new data to a file

In [ ]:
with open('newData.word', 'w') as wordWriter:
  with open('newData.tag', 'w') as tagWriter:
    for i in range(len(augmented_data)):
      wordWriter.write(" ".join(augmented_data[i][0]) + "\n")
      tagWriter.write(" ".join(augmented_data[i][1]) + "\n")

Append New Data

In [ ]:
new_training_data = []
new_tags = []
new_sentences = []
with open("newData.tag") as tagsData:
    with open("newData.word") as sentencesData:
        for line in tagsData:
            new_tags.append(line)
        for line in sentencesData:
            new_sentences.append(line)
            
for i in range(len(new_tags)):
  pair = (new_sentences[i].split(), new_tags[i].split())
  new_training_data.append(pair)

print(len(new_training_data))

11586


In [ ]:
augmented_training_data = copy.deepcopy(training_data)
print("Training data length before: {0}".format(len(augmented_training_data)))
for item in new_training_data:
    augmented_training_data.append(item)
print("Training data length after: {0}".format(len(augmented_training_data)))

Training data length before: 10800
Training data length after: 22386


In [ ]:
buckets = {}

for i in range(len(augmented_training_data)):
  for j in range(len(augmented_training_data[i][0])):
    if augmented_training_data[i][1][j] not in buckets:
      buckets[augmented_training_data[i][1][j]] = []
    buckets[augmented_training_data[i][1][j]].append(augmented_training_data[i][0][j])
total = 0
for key in buckets:
  total = total + len(buckets[key])
print(total)

for key in buckets:
  print("{0}: {1}, {2:.2f}%".format(key, len(buckets[key]), len(buckets[key]) / total * 100))

489761
NN: 137594, 28.09%
AUX: 11072, 2.26%
VB: 79299, 16.19%
JJ: 19888, 4.06%
CD: 15219, 3.11%
IN: 60334, 12.32%
PN: 42912, 8.76%
SYM: 23890, 4.88%
PRO: 37744, 7.71%
DT: 15974, 3.26%
RB: 28462, 5.81%
CC: 12646, 2.58%
PA: 4018, 0.82%
QT: 367, 0.07%
AB: 342, 0.07%


Add unknowns to training data

In [ ]:
x = random.randint(len(training_data), size=(NUM_RAND_UNK))
for i in x:
  y = random.randint(len(training_data[i][0]))
  training_data[i][0][y] = 'UNK'

aug_x = random.randint(len(augmented_training_data), size=(AUG_NUM_RAND_UNK))
for i in aug_x:
  y = random.randint(len(augmented_training_data[i][0]))
  augmented_training_data[i][0][y] = 'UNK'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long).to(device)

word_to_index = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_index:  # word has not been assigned an index yet
            word_to_index[word] = len(word_to_index)  # Assign each word with a unique index
print(len(word_to_index))

aug_word_to_index = {}
for sent, tags in augmented_training_data:
    for word in sent:
        if word not in aug_word_to_index:  # word has not been assigned an index yet
            aug_word_to_index[word] = len(aug_word_to_index)  # Assign each word with a unique index
print(len(aug_word_to_index))
# Assign each tag with a unique index
tag_to_index = {
    "AB": 0,
    "AUX": 1,
    "CC": 2,
    "CD": 3,
    "DT": 4,
    "IN": 5,
    "JJ": 6,
    "VB": 7,
    "NN": 8,
    "PN": 9,
    "PA": 10,
    "PRO": 11,
    "QT": 12,
    "RB": 13,
    "SYM": 14,
    }  

7331
7358


Create the model:

In [ ]:
class LSTMTagger(nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
    super(LSTMTagger, self).__init__()
    self.hidden_dim = hidden_dim

    self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
    
    # The LSTM takes word embeddings as inputs, and outputs hidden states
    # with dimensionality hidden_dim.
    self.lstm = nn.LSTM(embedding_dim, hidden_dim)

    # The linear layer that maps from hidden state space to tag space
    self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
  
  def forward(self, sentence):
    embeds = self.word_embeddings(sentence)
    lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
    tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
    tag_scores = F.log_softmax(tag_space, dim=1)
    return tag_scores

In [ ]:
for i in range(len(test_data)):
  for j in range(len(test_data[i][0])):
    if test_data[i][0][j] not in word_to_index:
      test_data[i][0][j] = 'UNK'

for i in range(len(close_test_data)):
  for j in range(len(close_test_data[i][0])):
    if close_test_data[i][0][j] not in word_to_index:
      close_test_data[i][0][j] = 'UNK'

for i in range(len(open_test_data)):
  for j in range(len(open_test_data[i][0])):
    if open_test_data[i][0][j] not in word_to_index:
      open_test_data[i][0][j] = 'UNK'

# For augmented test:
test_data_aug = copy.deepcopy(test_data)
for i in range(len(test_data_aug)):
  for j in range(len(test_data_aug[i][0])):
    if test_data_aug[i][0][j] not in aug_word_to_index:
      test_data_aug[i][0][j] = 'UNK'

close_test_data_aug = copy.deepcopy(close_test_data)
for i in range(len(close_test_data_aug)):
  for j in range(len(close_test_data_aug[i][0])):
    if close_test_data_aug[i][0][j] not in aug_word_to_index:
      close_test_data_aug[i][0][j] = 'UNK'

open_test_data_aug = copy.deepcopy(open_test_data)
for i in range(len(open_test_data_aug)):
  for j in range(len(open_test_data_aug[i][0])):
    if open_test_data_aug[i][0][j] not in aug_word_to_index:
      open_test_data_aug[i][0][j] = 'UNK'

In [ ]:
def evaluateAndPrintModel(epoch, startTime, model, word_to_index, test_data):
  allTags = {value:key for key, value in tag_to_index.items()}

  y_pred = []

  for i in range(len(test_data)):
    test_sentence = test_data[i][0]
    
    with torch.no_grad():
      inputs = prepare_sequence(test_sentence, word_to_index)
      tag_scores = model(inputs).cpu()

    for i in range(len(tag_scores)):
      index = np.argmin(abs(tag_scores[i])).item()
      y_pred.append(allTags.get(index))

  y_true = []

  for i in range(len(test_data)):
    for item in test_data[i][1]:
      y_true.append(item)

  a = accuracy_score(y_true, y_pred)
  p = precision_score(y_true, y_pred, average='macro', zero_division=1)
  r = recall_score(y_true, y_pred, average='macro', zero_division=1)
  F1 = 2 * (p * r) / (p + r)
  currentTimeMin = (int) ((time.time() - startTime)/60)
  currentTimeSec = (int) ((time.time() - startTime) % 60)
  output = "Epoch {3}\t\t A: {6:.4g} P: {0:.4g} R: {1:.4g} F: {2:.4g} in {4} minutes and {5} seconds\n".format(p, r, F1, epoch + 1, currentTimeMin, currentTimeSec, a)
  print(output)
  return output

Train Original Data Model:

In [ ]:
absoluteStartTime = time.time()

In [ ]:
fileLog = open("log_normal.txt", "w")
for i in range(25):
  print("Normal Data Model {0}".format(i))
  fileLog.write("Normal Data Model {0}\n".format(i))

  normal_model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_index), len(tag_to_index))
  normal_model = normal_model.to(device)
  loss_function = nn.NLLLoss()
  optimizer = optim.SGD(normal_model.parameters(), lr=LEARNING_RATE)

  with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_index)
    tag_scores = normal_model(inputs)

  startTime = time.time()

  for epoch in range(NUM_EPOCHS):
    #if epoch % 1 == 0: print("Epoch {0}  Time: {1} minutes".format(epoch, (time.time() - startTime)/60))

    for sentence, tags in training_data:
      # Step 1. Remember that Pytorch accumulates gradients.
      # We need to clear them out before each instance
      normal_model.zero_grad()

      # Step 2. Get our inputs ready for the network, that is, turn them into
      # Tensors of word indices.
      sentence_in = prepare_sequence(sentence, word_to_index)
      targets = prepare_sequence(tags, tag_to_index)

      # Step 3. Run our forward pass.
      tag_scores = normal_model(sentence_in)

      # Step 4. Compute the loss, gradients, and update the parameters by
      #  calling optimizer.step()
      loss = loss_function(tag_scores, targets)
      loss.backward()
      optimizer.step()

    #print("OPEN TEST DATA:")
    #f.write("OPEN TEST DATA:\t")
    #open_output = evaluateAndPrintModel(epoch, startTime, normal_model, word_to_index, open_test_data)
    #f.write(open_output)

    #print("CLOSE TEST DATA:")
    #f.write("CLOSE TEST DATA:\t")
    #close_output = evaluateAndPrintModel(epoch, startTime, normal_model, word_to_index, close_test_data)
    #f.write(close_output)

    #print("TEST SET DATA:")
    #f.write("TEST SET DATA:\t")
    test_data_output = evaluateAndPrintModel(epoch, startTime, normal_model, word_to_index, test_data)
    fileLog.write(test_data_output)

  print("Done!")
  print("Total time: {0} minutes".format((time.time() - startTime)/60))
  fileLog.write("Total time: {0} minutes\n\n".format((time.time() - startTime)/60))
fileLog.close()

Streaming output truncated to the last 5000 lines.
Epoch 214		 A: 0.9548 P: 0.9471 R: 0.9355 F: 0.9413 in 60 minutes and 43 seconds

Epoch 215		 A: 0.9548 P: 0.9471 R: 0.9355 F: 0.9413 in 61 minutes and 0 seconds

Epoch 216		 A: 0.9549 P: 0.9473 R: 0.9357 F: 0.9414 in 61 minutes and 17 seconds

Epoch 217		 A: 0.9549 P: 0.9473 R: 0.9357 F: 0.9415 in 61 minutes and 34 seconds

Epoch 218		 A: 0.9549 P: 0.9473 R: 0.9357 F: 0.9415 in 61 minutes and 51 seconds

Epoch 219		 A: 0.9549 P: 0.9473 R: 0.9357 F: 0.9415 in 62 minutes and 8 seconds

Epoch 220		 A: 0.955 P: 0.9473 R: 0.9358 F: 0.9416 in 62 minutes and 25 seconds

Epoch 221		 A: 0.9549 P: 0.9473 R: 0.9358 F: 0.9415 in 62 minutes and 42 seconds

Epoch 222		 A: 0.9549 P: 0.9473 R: 0.9358 F: 0.9415 in 62 minutes and 59 seconds

Epoch 223		 A: 0.955 P: 0.9472 R: 0.9359 F: 0.9415 in 63 minutes and 16 seconds

Epoch 224		 A: 0.955 P: 0.9472 R: 0.9359 F: 0.9415 in 63 minutes and 33 seconds

Epoch 225		 A: 0.955 P: 0.9472 R: 0.9359 F: 0.9415 i

Train Augmented Data Model:

In [ ]:
f = open("log_augment.txt", "w")
for i in range(25):
  print("Augmented Data Model {0}".format(i))
  f.write("Augmented Data Model {0}\n".format(i))

  aug_model = LSTMTagger(AUG_EMBEDDING_DIM, AUG_HIDDEN_DIM, len(aug_word_to_index), len(tag_to_index))
  aug_model = aug_model.to(device)
  loss_function = nn.NLLLoss()
  optimizer = optim.SGD(aug_model.parameters(), lr=LEARNING_RATE)

  with torch.no_grad():
    inputs = prepare_sequence(augmented_training_data[0][0], aug_word_to_index)
    tag_scores = aug_model(inputs)

  startTime = time.time()

  for epoch in range(AUG_NUM_EPOCHS):
    #if epoch % 1 == 0: print("Epoch {0}  Time: {1} minutes".format(epoch, (time.time() - startTime)/60))

    for sentence, tags in augmented_training_data:
      # Step 1. Remember that Pytorch accumulates gradients.
      # We need to clear them out before each instance
      aug_model.zero_grad()

      # Step 2. Get our inputs ready for the network, that is, turn them into
      # Tensors of word indices.
      sentence_in = prepare_sequence(sentence, aug_word_to_index)
      targets = prepare_sequence(tags, tag_to_index)

      # Step 3. Run our forward pass.
      tag_scores = aug_model(sentence_in)

      # Step 4. Compute the loss, gradients, and update the parameters by
      #  calling optimizer.step()
      loss = loss_function(tag_scores, targets)
      loss.backward()
      optimizer.step()

    #print("OPEN TEST DATA:")
    #f.write("OPEN TEST DATA:\t")
    #open_output = evaluateAndPrintModel(epoch, startTime, aug_model, aug_word_to_index, open_test_data_aug)
    #f.write(open_output)

    #print("CLOSE TEST DATA:")
    #f.write("CLOSE TEST DATA:\t")
    #close_output = evaluateAndPrintModel(epoch, startTime, aug_model, aug_word_to_index, close_test_data_aug)
    #f.write(close_output)

    #print("TEST SET DATA:")
    #f.write("TEST SET DATA:\t")
    test_data_output = evaluateAndPrintModel(epoch, startTime, aug_model, aug_word_to_index, test_data_aug)
    f.write(test_data_output)

  print("Done!")
  print("Total time: {0} minutes".format((time.time() - startTime)/60))
  f.write("Total time: {0} minutes\n\n".format((time.time() - startTime)/60))
f.close()

Augmented Data Model 0
Epoch 1		 A: 0.7078 P: 0.8963 R: 0.4689 F: 0.6157 in 0 minutes and 42 seconds

Epoch 2		 A: 0.7975 P: 0.9135 R: 0.5759 F: 0.7065 in 1 minutes and 25 seconds

Epoch 3		 A: 0.8419 P: 0.9224 R: 0.6415 F: 0.7567 in 2 minutes and 7 seconds

Epoch 4		 A: 0.8697 P: 0.9285 R: 0.6819 F: 0.7863 in 2 minutes and 50 seconds

Epoch 5		 A: 0.8897 P: 0.9345 R: 0.7124 F: 0.8085 in 3 minutes and 32 seconds

Epoch 6		 A: 0.9035 P: 0.9376 R: 0.7729 F: 0.8473 in 4 minutes and 15 seconds

Epoch 7		 A: 0.9135 P: 0.9393 R: 0.7862 F: 0.856 in 4 minutes and 57 seconds

Epoch 8		 A: 0.9208 P: 0.9425 R: 0.7966 F: 0.8634 in 5 minutes and 40 seconds

Epoch 9		 A: 0.9263 P: 0.9449 R: 0.8307 F: 0.8841 in 6 minutes and 22 seconds

Epoch 10		 A: 0.9311 P: 0.9478 R: 0.8382 F: 0.8896 in 7 minutes and 5 seconds

Epoch 11		 A: 0.9343 P: 0.9483 R: 0.8431 F: 0.8926 in 7 minutes and 47 seconds

Epoch 12		 A: 0.9368 P: 0.9507 R: 0.8472 F: 0.896 in 8 minutes and 30 seconds

Epoch 13		 A: 0.9395 P: 0.9498

In [ ]:
for key, value in aug_word_to_index.items():
  if key == 'ខ្ញុំ':
    print("Found")

In [ ]:
currentTimeHour = (int) (((time.time() - absoluteStartTime)/60/60))
currentTimeMin = (int) ((time.time() - absoluteStartTime)/60 % 60)
currentTimeSec = (int) ((time.time() - absoluteStartTime) % 60)
print("Total time: {0}:{1}:{2}".format(currentTimeHour, currentTimeMin, currentTimeSec))

Total time: 68:44:59
